In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/msl-lzq/MSL_train.npy
/kaggle/input/msl-lzq/MSL_test.npy
/kaggle/input/msl-lzq/MSL_test_label.npy


In [9]:
# The following is from base file:



##################################################################

from abc import ABC, abstractmethod # abc是一个抽象子类，现在有点难理解， 问问别人 放两个链接在这儿 # https://docs.python.org/zh-cn/3/library/abc.html # https://blog.csdn.net/weixin_40907382/article/details/80277170
from torch.utils.data import DataLoader



import torch
import os
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import collections
import numbers
import math
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pickle




# def get_loader_segment(data_path, batch_size, win_size=100, step=100, mode='train', dataset='MSL'):
#     dataset = MSLSegLoader(data_path, win_size, 1, mode)

#     shuffle = False
#     if mode == 'train':
#         shuffle = True

#     data_loader = DataLoader(dataset=dataset,
#                              batch_size=batch_size,
#                              shuffle=shuffle,
#                              num_workers=0)
#     return data_loader



class BaseADDataset(ABC):
    """Anomaly detection dataset base class."""
    
    """为了加载数据的一个类，需要有区分 看看后面怎么改"""
    
    def __init__(self, root: str):
        super().__init__()
        self.root = root  # root path to data
        
  # 代码这里选择了两种类

        self.n_classes = 2  # 0: normal, 1: outlier
        self.normal_classes = None  # tuple with original class labels that define the normal class
        self.outlier_classes = None  # tuple with original class labels that define the outlier class

        self.train_set = None  # must be of type torch.utils.data.Dataset ！！！！！！！一定要是这个类型的数据
        self.test_set = None  # must be of type torch.utils.data.Dataset

        
    
    @abstractmethod
    def loaders(self, batch_size: int, shuffle_train=True, shuffle_test=False, num_workers: int = 0) -> (DataLoader, DataLoader):
        """Implement data loaders of type torch.utils.data.DataLoader for train_set and test_set."""
        pass

    def __repr__(self):
        return self.__class__.__name__

    
    
    
##################################################################
import logging  # 日志模块
# logging模块定义的函数和类为应用程序和库的开发实现了一个灵活的事件日志系统。
# logging模块是Python的一个标准库模块，由标准库模块提供日志记录API的关键好处是所有Python模块都可以使用这个日志记录功能。
# 所以，你的应用日志可以将你自己的日志信息与来自第三方模块的信息整合起来。
import torch.nn as nn
import numpy as np


class BaseNet(nn.Module):
    """Base class for all neural networks."""

    def __init__(self):
        super().__init__()
        self.logger = logging.getLogger(self.__class__.__name__) # 得到一个Logger对象
        self.rep_dim = None  # representation dimensionality, i.e. dim of the last layer

    def forward(self, *input):
        """
        Forward pass logic
        :return: Network output
        """
        raise NotImplementedError

    def summary(self):
        """Network summary."""
        net_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in net_parameters])
        
        """logger.info 创建一个与它们的方法名对应等级的日志记录"""
        self.logger.info('Trainable parameters: {}'.format(params))
        self.logger.info(self)
      
    
    
##################################################################
from abc import ABC, abstractmethod
"""BaseTrainer 创建一个最基本的训练器，为后面提供便捷"""
class BaseTrainer(ABC):
    """Trainer base class."""

    def __init__(self, optimizer_name: str, lr: float, n_epochs: int, lr_milestones: tuple, batch_size: int,
                 weight_decay: float, device: str, n_jobs_dataloader: int):
        super().__init__()
        self.optimizer_name = optimizer_name # 挑一个最喜欢的强化器
        self.lr = lr # learning rate
        self.n_epochs = n_epochs # 几层训练
        self.lr_milestones = lr_milestones # 没懂
        self.batch_size = batch_size 
        self.weight_decay = weight_decay # SVDD 的权重矩阵，但是不知道用在哪儿？？？
        self.device = device # 到时候用gpu
        self.n_jobs_dataloader = n_jobs_dataloader # 没懂
        
        
    """ abstractmethod 相当于一个接口
    
        子类必须全部实现重写父类的abstractmethod方法
        
        非abstractmethod方法可以不实现重写
        
        带abstractmethod方法的类不能实例化
        
        链接 https://blog.csdn.net/u013210620/article/details/78604077
    """


    @abstractmethod
    def train(self, dataset: BaseADDataset, net: BaseNet) -> BaseNet:
        """
        Implement train method that trains the given network using the train_set of dataset.
        :return: Trained net
        """
        pass

    @abstractmethod
    def test(self, dataset: BaseADDataset, net: BaseNet):
        """
        Implement test method that evaluates the test_set of dataset on the given network.
        """
        pass

    
    
##################################################################
from torch.utils.data import DataLoader
""" DataLoader：
    就是数据加载器，结合了数据集和取样器，并且可以提供多个线程处理数据集。
    
    在训练模型时使用到此函数，用来把训练数据分成多个小组，此函数每次抛出一组数据。
    
    直至把所有的数据都抛出。就是做一个数据的初始化。
"""

class TorchvisionDataset(BaseADDataset):
    """TorchvisionDataset class for datasets already implemented in torchvision.datasets."""

    def __init__(self, root: str):
        super().__init__(root)
    
#     def loaders(self, batch_size: int, shuffle_train=True, shuffle_test=False, num_workers: int = 0) -> (DataLoader, DataLoader):
#         train_loader = DataLoader(dataset=self.train_set, batch_size=batch_size, shuffle=shuffle_train,
#                                   num_workers=num_workers)
#         test_loader = DataLoader(dataset=self.test_set, batch_size=batch_size, shuffle=shuffle_test,
#                                  num_workers=num_workers)
#         return train_loader, test_loader
    
    
    def loaders(self, batch_size: int, shuffle_train=True, shuffle_test=False, num_workers: int = 0) -> (DataLoader, DataLoader):
        

#         shuffle = False
#         if mode == 'train':
        shuffle = True
#         print(self.train_set)
        data_loader = DataLoader(dataset=self.train_set,
                                 batch_size=batch_size,
                                 shuffle=shuffle_train,
                                 num_workers=0)
        return data_loader

class CIFAR10_Dataset(TorchvisionDataset):

    def __init__(self, root: str, normal_class=5):
        super().__init__(root)

        self.n_classes = 2  # 0: normal, 1: outlier
        self.normal_classes = tuple([normal_class])
        self.outlier_classes = list(range(0, 10))

        
        self.train_set = MSLSegLoader(data_path = '../input/msl-lzq', win_size = 100, step = 1, mode = 'train')
#         self.train_set = self.train_set.unsqueeze(0)
        self.test_set = MSLSegLoader(data_path = '../input/msl-lzq', win_size = 100, step = 1, mode = 'test')
#         self.test_set = self.test_set.unsqueeze(0)
    
class MSLSegLoader(object):
    def __init__(self, data_path, win_size, step, mode="train"):
        self.mode = mode
        self.step = step
        self.win_size = win_size
        self.scaler = StandardScaler()
        data = np.load(data_path + "/MSL_test.npy")
        self.scaler.fit(data)
        data = self.scaler.transform(data)
        test_data = np.load(data_path + "/MSL_test.npy")
        self.test = self.scaler.transform(test_data)

        self.train = data
        
        self.val = self.test
        self.test_labels = np.load(data_path + "/MSL_test_label.npy")
#         print("test:", self.test.shape)
#         print("train:", self.train.shape)

    def __len__(self):

        if self.mode == "train":
            return (self.train.shape[0] - self.win_size) // self.step + 1
        elif (self.mode == 'val'):
            return (self.val.shape[0] - self.win_size) // self.step + 1
        elif (self.mode == 'test'):
            return (self.test.shape[0] - self.win_size) // self.step + 1
        else:
            return (self.test.shape[0] - self.win_size) // self.win_size + 1

    def __getitem__(self, index):
        index = index * self.step
        if self.mode == "train":
#             print('aaaaaaa', type(np.float32(self.train[index:index + self.win_size])))
#             print('bbbbbbb', type( np.float32(self.test_labels[0:self.win_size])))
#             print('aaaaaaa', np.float32(self.train[index:index + self.win_size]))
#             print('bbbbbbb',  np.float32(self.test_labels[0:self.win_size]))
            Train = np.float32(self.train[index:index + self.win_size])
#             print(Train.shape)
#             Train = np.reshape(Train,(1,100,55))
            return Train, np.float32(self.test_labels[0:self.win_size])
        elif (self.mode == 'val'):
            return np.float32(self.val[index:index + self.win_size]), np.float32(self.test_labels[0:self.win_size])
        elif (self.mode == 'test'):
            Test = np.float32(self.test[index:index + self.win_size])
#             Test = np.reshape(Test,(1,100,55))
            return Test, np.float32(self.test_labels[index:index + self.win_size])
        else:
            return np.float32(self.test[
                              index // self.step * self.win_size:index // self.step * self.win_size + self.win_size]), np.float32(
                self.test_labels[index // self.step * self.win_size:index // self.step * self.win_size + self.win_size])



In [10]:
# The following is from optim file:


##################################################################
from sklearn.metrics import roc_auc_score
import logging
import time
import torch
import torch.optim as optim
import numpy as np


# 从这里调用了 BaseTrainer
class AETrainer(BaseTrainer):

    """
    
    所有的设置都在这里改！！！！！！！！
    我先降低了“n_epochs“ = 5
    """
    
    def __init__(self, optimizer_name: str = 'adam', lr: float = 0.001, n_epochs: int = 5, lr_milestones: tuple = (),
                 batch_size: int = 128, weight_decay: float = 1e-6, device: str = 'cuda', n_jobs_dataloader: int = 0):
        
        
        super().__init__(optimizer_name, lr, n_epochs, lr_milestones, batch_size, weight_decay, device, n_jobs_dataloader)

    def train(self, dataset: BaseADDataset, ae_net: BaseNet):
        logger = logging.getLogger()

        # Set device for network 
        """ 调用GPU """
        ae_net = ae_net.to(self.device)

        
        # Get train data loader
        """ 读取训练集数据 """
        train_loader = dataset.loaders(batch_size=self.batch_size, num_workers=self.n_jobs_dataloader)

        
        # Set optimizer (Adam optimizer for now)
        """ 设置优化器！！！！！！！！！ """
        optimizer = optim.Adam(ae_net.parameters(), lr=self.lr, weight_decay=self.weight_decay,
                               amsgrad=self.optimizer_name == 'amsgrad')

        
        # Set learning rate scheduler
        """ 设置学习率。。。 """
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.lr_milestones, gamma=0.1)

        
        # Training
        logger.info('Starting pretraining...')
        start_time = time.time()
        ae_net.train()
        for epoch in range(self.n_epochs):

            scheduler.step()
            if epoch in self.lr_milestones:
                logger.info('  LR scheduler: new learning rate is %g' % float(scheduler.get_lr()[0]))

            loss_epoch = 0.0
            n_batches = 0
            epoch_start_time = time.time()
            for data in train_loader:
                try:
                    inputs, _, _ = data
                except:
                    inputs, _ = data
                """"""
                    
#                 print('input shape:', inputs.shape)
#                 print('input type:', type(inputs))
                
                
                """"""
                inputs = inputs.to(self.device)

                # Zero the network parameter gradients
                optimizer.zero_grad()

                # Update network parameters via backpropagation: forward + backward + optimize
                outputs = ae_net(inputs)
#                 print('input shape:', inputs.shape)
#                 print('output shape:', outputs.shape)
                scores = torch.sum((outputs - inputs) ** 2, dim=tuple(range(1, outputs.dim())))
                loss = torch.mean(scores)
                loss.backward()
                optimizer.step()

                loss_epoch += loss.item()
                n_batches += 1

            # log epoch statistics
            epoch_train_time = time.time() - epoch_start_time
            logger.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f}'
                        .format(epoch + 1, self.n_epochs, epoch_train_time, loss_epoch / n_batches))

        pretrain_time = time.time() - start_time
        logger.info('Pretraining time: %.3f' % pretrain_time)
        logger.info('Finished pretraining.')

        return ae_net

    def test(self, dataset: BaseADDataset, ae_net: BaseNet):
        logger = logging.getLogger()

        # Set device for network
        ae_net = ae_net.to(self.device)

        # Get test data loader
        test_loader = dataset.loaders(batch_size=self.batch_size, num_workers=self.n_jobs_dataloader)

        # Testing
        logger.info('Testing autoencoder...')
        loss_epoch = 0.0
        n_batches = 0
        start_time = time.time()
        idx_label_score = []
        ae_net.eval()
        with torch.no_grad():
            for data in test_loader:
                # 这里label后删除了idx！！！！！！
                inputs, labels = data
                inputs = inputs.to(self.device)
                outputs = ae_net(inputs)
                scores = torch.sum((outputs - inputs) ** 2, dim=tuple(range(1, outputs.dim())))
                loss = torch.mean(scores)

                # Save triple of (idx, label, score) in a ， 这里删除了 idx： idx.cpu().data.numpy().tolist(),
                idx_label_score += list(zip(
                                            labels.cpu().data.numpy().tolist(),
                                            scores.cpu().data.numpy().tolist()))

                loss_epoch += loss.item()
                n_batches += 1

        logger.info('Test set Loss: {:.8f}'.format(loss_epoch / n_batches))
        # 这里删除了 idx： _, 
        labels, scores = zip(*idx_label_score)
        labels = np.array(labels)
        scores = np.array(scores)
        print("lables: ", labels)
        print("scores: ", scores)
#         auc = roc_auc_score(labels, scores)
#         logger.info('Test set AUC: {:.2f}%'.format(100. * auc))

        test_time = time.time() - start_time
        logger.info('Autoencoder testing time: %.3f' % test_time)
        logger.info('Finished testing autoencoder.')

        
        
        
        
        
##############################################################################
from torch.utils.data.dataloader import DataLoader
from sklearn.metrics import roc_auc_score

import logging
import time
import torch
import torch.optim as optim
import numpy as np

""" n_epochs 被调成了5！！！！ """


class DeepSVDDTrainer(BaseTrainer):

    def __init__(self, objective, R, c, nu: float, optimizer_name: str = 'adam', lr: float = 0.001, n_epochs: int = 5,
                 lr_milestones: tuple = (), batch_size: int = 128, weight_decay: float = 1e-6, device: str = 'cuda',
                 n_jobs_dataloader: int = 0):
        super().__init__(optimizer_name, lr, n_epochs, lr_milestones, batch_size, weight_decay, device,
                         n_jobs_dataloader)
        
        
        """ 
        选择是单分类还是 软边界，应该是基于train集是否有标签？？？
        
        """

        assert objective in ('one-class', 'soft-boundary'), "Objective must be either 'one-class' or 'soft-boundary'."
        self.objective = objective

        """ 
        
        Deep SVDD parameters
        
        设置Deep SVDD的半径，圆心和 “nu”---不知道是啥 ？？？
        
        """
        # Deep SVDD parameters
        self.R = torch.tensor(R, device=self.device)  # radius R initialized with 0 by default.
        self.c = torch.tensor(c, device=self.device) if c is not None else None
        self.nu = nu

        
        """ 
        
        Optimization parameters
        
        在半径更新前，跑几个epochs
        
        """
#         self.warm_up_n_epochs = 10  # number of training epochs for soft-boundary Deep SVDD before radius R gets updated
        self.warm_up_n_epochs = 1 # for test
        # Results
        self.train_time = None
        self.test_auc = None
        self.test_time = None
        self.test_scores = None

        
    """
    
    最终的train函数
    
    参考上文标注
    
    """
    
    def train(self, dataset: BaseADDataset, net: BaseNet):
        logger = logging.getLogger()

        # Set device for network
        net = net.to(self.device)

        # Get train data loader
        train_loader = dataset.loaders(batch_size=self.batch_size, num_workers=self.n_jobs_dataloader)

        # Set optimizer (Adam optimizer for now)
        optimizer = optim.Adam(net.parameters(), lr=self.lr, weight_decay=self.weight_decay,
                               amsgrad=self.optimizer_name == 'amsgrad')

        # Set learning rate scheduler
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.lr_milestones, gamma=0.1)

        # Initialize hypersphere center c (if c not loaded)
        if self.c is None:
            logger.info('Initializing center c...')
            self.c = self.init_center_c(train_loader, net)
            logger.info('Center c initialized.')

        # Training
        logger.info('Starting training...')
        start_time = time.time()
        net.train()
        for epoch in range(self.n_epochs):

            scheduler.step()
            if epoch in self.lr_milestones:
                logger.info('  LR scheduler: new learning rate is %g' % float(scheduler.get_lr()[0]))

            loss_epoch = 0.0
            n_batches = 0
            epoch_start_time = time.time()
            for data in train_loader:
                try:
                    inputs, _, _ = data
                except:
                    inputs, _ = data
#                 print(inputs)
                inputs = inputs.to(self.device)

                # Zero the network parameter gradients
                optimizer.zero_grad()

                # Update network parameters via backpropagation: forward + backward + optimize
                outputs = net(inputs)
                dist = torch.sum((outputs - self.c) ** 2, dim=1)
                if self.objective == 'soft-boundary':
                    scores = dist - self.R ** 2
                    loss = self.R ** 2 + (1 / self.nu) * torch.mean(torch.max(torch.zeros_like(scores), scores))
                else:
                    loss = torch.mean(dist)
                loss.backward()
                optimizer.step()

                # Update hypersphere radius R on mini-batch distances
                if (self.objective == 'soft-boundary') and (epoch >= self.warm_up_n_epochs):
                    self.R.data = torch.tensor(get_radius(dist, self.nu), device=self.device)

                loss_epoch += loss.item()
                n_batches += 1

            # log epoch statistics 记录一下训练过程
            epoch_train_time = time.time() - epoch_start_time
            logger.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f}'
                        .format(epoch + 1, self.n_epochs, epoch_train_time, loss_epoch / n_batches))

        self.train_time = time.time() - start_time
        logger.info('Training time: %.3f' % self.train_time)

        logger.info('Finished training.')

        return net

    def test(self, dataset: BaseADDataset, net: BaseNet):
        logger = logging.getLogger()

        # Set device for network
        net = net.to(self.device)

        # Get test data loader
        test_loader = dataset.loaders(batch_size=self.batch_size, num_workers=self.n_jobs_dataloader)

        # Testing
        logger.info('Starting testing...')
        start_time = time.time()
        idx_label_score = []
        net.eval()
        with torch.no_grad():
            for data in test_loader:
                # 这里label后删除了idx！！！！！！
                inputs, labels = data
                inputs = inputs.to(self.device)
                outputs = net(inputs)
                dist = torch.sum((outputs - self.c) ** 2, dim=1)
                if self.objective == 'soft-boundary':
                    scores = dist - self.R ** 2
                else:
                    scores = dist

                # Save triples of (idx, label, score) in a list, 这里把 idx 给删除了
                
                
                idx_label_score += list(zip(
                                            labels.cpu().data.numpy().tolist(),
                                            scores.cpu().data.numpy().tolist()))

        self.test_time = time.time() - start_time
        logger.info('Testing time: %.3f' % self.test_time)

        self.test_scores = idx_label_score

        # Compute AUC 这里删除了 idx： _,
        labels, scores = zip(*idx_label_score)
        labels = np.array(labels)
        scores = np.array(scores)
        
#         print("lables: ", labels)
#         print("scores: ", scores)

#         self.test_auc = roc_auc_score(labels, scores)
#         logger.info('Test set AUC: {:.2f}%'.format(100. * self.test_auc))

        logger.info('Finished testing.')

        
        
    def init_center_c(self, train_loader: DataLoader, net: BaseNet, eps=0.1):
        
        """
        
        用均值初始化圆心
        
        Initialize hypersphere center c as the mean from an initial forward pass on the data.
        
        """
        
        n_samples = 0
        
        ''' 修改圆心为2维'''
        
#         c = torch.zeros(net.rep_dim, device=self.device)
        c = torch.zeros([100,3], device=self.device)

        net.eval()
        with torch.no_grad():
            for data in train_loader:
                # get the inputs of the batch
                try:
                    inputs, _, _ = data
                except:
                    inputs, _ = data
                inputs = inputs.to(self.device)
                outputs = net(inputs)
#                 print('output: ', outputs)
#                 print('output shape: ', outputs.shape)

                n_samples += outputs.shape[0]
                c += torch.sum(outputs, dim=0)

        c /= n_samples

        # If c_i is too close to 0, set to +-eps. Reason: a zero unit can be trivially matched with zero weights.
        c[(abs(c) < eps) & (c < 0)] = -eps
        c[(abs(c) < eps) & (c > 0)] = eps

        return c


    
def get_radius(dist: torch.Tensor, nu: float):
    
    """
    
    计算半径
    
    Optimally solve for radius R via the (1-nu)-quantile of distances.
    
    """
    return np.quantile(np.sqrt(dist.clone().data.cpu().numpy()), 1 - nu)


In [11]:
# the full functions of Deep_SVDD

import json
import torch

class DeepSVDD(object):
    """A class for the Deep SVDD method.

    Attributes:
        objective: A string specifying the Deep SVDD objective (either 'one-class' or 'soft-boundary').
        nu: Deep SVDD hyperparameter nu (must be 0 < nu <= 1).
        R: Hypersphere radius R.
        c: Hypersphere center c.
        net_name: A string indicating the name of the neural network to use.
        net: The neural network \phi.
        ae_net: The autoencoder network corresponding to \phi for network weights pretraining.
        trainer: DeepSVDDTrainer to train a Deep SVDD model.
        optimizer_name: A string indicating the optimizer to use for training the Deep SVDD network.
        ae_trainer: AETrainer to train an autoencoder in pretraining.
        ae_optimizer_name: A string indicating the optimizer to use for pretraining the autoencoder.
        results: A dictionary to save the results.
    """

    def __init__(self, objective: str = 'one-class', nu: float = 0.1):
        """Inits DeepSVDD with one of the two objectives and hyperparameter nu."""

        assert objective in ('one-class', 'soft-boundary'), "Objective must be either 'one-class' or 'soft-boundary'."
        self.objective = objective
        assert (0 < nu) & (nu <= 1), "For hyperparameter nu, it must hold: 0 < nu <= 1."
        self.nu = nu
        self.R = 0.0  # hypersphere radius R
        self.c = None  # hypersphere center c

        self.net_name = None
        self.net = None  # neural network \phi

        self.trainer = None
        self.optimizer_name = None

        self.ae_net = None  # autoencoder network for pretraining
        self.ae_trainer = None
        self.ae_optimizer_name = None

        self.results = {
            'train_time': None,
            'test_auc': None,
            'test_time': None,
            'test_scores': None,
        }

    def set_network(self, net_name):
        """Builds the neural network \phi."""
        self.net_name = net_name
        self.net = build_network(net_name)

        
        """ n_epochs = 50 """
        
    def train(self, dataset: BaseADDataset, optimizer_name: str = 'adam', lr: float = 0.001, n_epochs: int = 5,
              lr_milestones: tuple = (), batch_size: int = 128, weight_decay: float = 1e-6, device: str = 'cuda',
              n_jobs_dataloader: int = 0):
        """Trains the Deep SVDD model on the training data."""

        self.optimizer_name = optimizer_name
        self.trainer = DeepSVDDTrainer(self.objective, self.R, self.c, self.nu, optimizer_name, lr=lr,
                                       n_epochs=n_epochs, lr_milestones=lr_milestones, batch_size=batch_size,
                                       weight_decay=weight_decay, device=device, n_jobs_dataloader=n_jobs_dataloader)
        # Get the model
        self.net = self.trainer.train(dataset, self.net)
        self.R = float(self.trainer.R.cpu().data.numpy())  # get float
        self.c = self.trainer.c.cpu().data.numpy().tolist()  # get list
        self.results['train_time'] = self.trainer.train_time

    def test(self, dataset: BaseADDataset, device: str = 'cuda', n_jobs_dataloader: int = 0):
        """Tests the Deep SVDD model on the test data."""

        if self.trainer is None:
            self.trainer = DeepSVDDTrainer(self.objective, self.R, self.c, self.nu,
                                           device=device, n_jobs_dataloader=n_jobs_dataloader)

        self.trainer.test(dataset, self.net)
        # Get results
        self.results['test_auc'] = self.trainer.test_auc
        self.results['test_time'] = self.trainer.test_time
        self.results['test_scores'] = self.trainer.test_scores

        
    """
    
    提前为Deep SVDD 训练一个 权重
    n_epochs: int = 100
    
    """
    
    def pretrain(self, dataset: BaseADDataset, optimizer_name: str = 'adam', lr: float = 0.001, n_epochs: int = 5,
                 lr_milestones: tuple = (), batch_size: int = 128, weight_decay: float = 1e-6, device: str = 'cuda',
                 n_jobs_dataloader: int = 0):
        """Pretrains the weights for the Deep SVDD network \phi via autoencoder."""

        self.ae_net = build_autoencoder(self.net_name)
#         print(self.ae_net.shape)
        self.ae_optimizer_name = optimizer_name
        self.ae_trainer = AETrainer(optimizer_name, lr=lr, n_epochs=n_epochs, lr_milestones=lr_milestones,
                                    batch_size=batch_size, weight_decay=weight_decay, device=device,
                                    n_jobs_dataloader=n_jobs_dataloader)
        
        
        """"""
        self.ae_net = self.ae_trainer.train(dataset, self.ae_net)
        """"""
        
        
        self.ae_trainer.test(dataset, self.ae_net)
        self.init_network_weights_from_pretraining()

        
    def init_network_weights_from_pretraining(self):
        
        """Initialize the Deep SVDD network weights from the encoder weights of the pretraining autoencoder."""

        net_dict = self.net.state_dict()
        ae_net_dict = self.ae_net.state_dict()

        # Filter out decoder network keys
        ae_net_dict = {k: v for k, v in ae_net_dict.items() if k in net_dict}
        # Overwrite values in the existing state_dict
        net_dict.update(ae_net_dict)
        # Load the new state_dict
        self.net.load_state_dict(net_dict)

        
    def save_model(self, export_model, save_ae=True):
        
        """Save Deep SVDD model to export_model."""

        net_dict = self.net.state_dict()
        ae_net_dict = self.ae_net.state_dict() if save_ae else None

        torch.save({'R': self.R,
                    'c': self.c,
                    'net_dict': net_dict,
                    'ae_net_dict': ae_net_dict}, export_model)

    def load_model(self, model_path, load_ae=False):
        
        """Load Deep SVDD model from model_path."""

        model_dict = torch.load(model_path)

        self.R = model_dict['R']
        self.c = model_dict['c']
        self.net.load_state_dict(model_dict['net_dict'])
        if load_ae:
            if self.ae_net is None:
                self.ae_net = build_autoencoder(self.net_name)
            self.ae_net.load_state_dict(model_dict['ae_net_dict'])

            
    def save_results(self, export_json):
        """Save results dict to a JSON-file."""
        with open(export_json, 'w') as fp:
            json.dump(self.results, fp)


In [12]:
# net work and autoencoder
import torch.nn.functional as F

class CIFAR10_LeNet_ELU(BaseNet):

    def __init__(self):
        super().__init__()

        self.rep_dim = 3
#         self.pool = nn.MaxPool2d(2, 2)

#         self.conv1 = nn.Conv2d(1, 32, 5, bias=False, padding=2)
#         self.bn2d1 = nn.BatchNorm2d(32, eps=1e-04, affine=False)
#         self.conv2 = nn.Conv2d(32, 64, 5, bias=False, padding=2)
#         self.bn2d2 = nn.BatchNorm2d(64, eps=1e-04, affine=False)
#         self.conv3 = nn.Conv2d(64, 128, 5, bias=False, padding=2)
#         self.bn2d3 = nn.BatchNorm2d(128, eps=1e-04, affine=False)
#         self.fc1 = nn.Linear(128 * 4 * 4, self.rep_dim, bias=False)
        self.encoder1 = nn.Sequential(
            nn.Linear(55, 48),
            nn.ReLU(True),
            nn.Linear(48, 32),
            nn.ReLU(True),
            nn.Linear(32, 12),
            nn.ReLU(True), 
            nn.Linear(12, 3))
        
#         self.linear1 = nn.Linear(100 * 55, 10 * 5)

        

    def forward(self, x):
#         x = self.conv1(x)
#         x = self.pool(F.elu(self.bn2d1(x)))
#         x = self.conv2(x)
#         x = self.pool(F.elu(self.bn2d2(x)))
#         x = self.conv3(x)
#         x = self.pool(F.elu(self.bn2d3(x)))
#         x = x.view(x.size(0), -1)
#         x = self.fc1(x)


        x = self.encoder1(x)
        return x


class CIFAR10_LeNet_ELU_Autoencoder(BaseNet):

    def __init__(self):
        super().__init__()

        self.rep_dim = 3
#         self.pool = nn.MaxPool2d(2,2)
#         # Encoder (must match the Deep SVDD network above)
#         self.conv1 = nn.Conv2d(1, 32, 5, bias=False, padding=2)
#         nn.init.xavier_uniform_(self.conv1.weight)
#         self.bn2d1 = nn.BatchNorm2d(32, eps=1e-04, affine=False)
#         self.conv2 = nn.Conv2d(32, 64, 5, bias=False, padding=2)
#         nn.init.xavier_uniform_(self.conv2.weight)
#         self.bn2d2 = nn.BatchNorm2d(64, eps=1e-04, affine=False)
#         self.conv3 = nn.Conv2d(64, 128, 5, bias=False, padding=2)
#         nn.init.xavier_uniform_(self.conv3.weight)
#         self.bn2d3 = nn.BatchNorm2d(128, eps=1e-04, affine=False)
#         self.fc1 = nn.Linear(9216, self.rep_dim, bias=False)
#         self.bn1d = nn.BatchNorm1d(self.rep_dim, eps=1e-04, affine=False)



#         # Decoder
#         self.deconv1 = nn.ConvTranspose2d(int(self.rep_dim / (4 * 4)), 128, 5, bias=False, padding=2)
#         nn.init.xavier_uniform_(self.deconv1.weight)
#         self.bn2d4 = nn.BatchNorm2d(128, eps=1e-04, affine=False)
#         self.deconv2 = nn.ConvTranspose2d(128, 64, 5, bias=False, padding=2)
#         nn.init.xavier_uniform_(self.deconv2.weight)
#         self.bn2d5 = nn.BatchNorm2d(64, eps=1e-04, affine=False)
#         self.deconv3 = nn.ConvTranspose2d(64, 32, 5, bias=False, padding=2)
#         nn.init.xavier_uniform_(self.deconv3.weight)
#         self.bn2d6 = nn.BatchNorm2d(32, eps=1e-04, affine=False)
#         self.deconv4 = nn.ConvTranspose2d(32, 1, 5, bias=False, padding=2)
#         nn.init.xavier_uniform_(self.deconv4.weight)
        
    
        self.encoder = nn.Sequential(
            nn.Linear(55, 48),
            nn.ReLU(True),
            nn.Linear(48, 32),
            nn.ReLU(True),
            nn.Linear(32, 12),
            nn.ReLU(True), 
            nn.Linear(12, 3))
        
        
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 32),
            nn.ReLU(True),
            nn.Linear(32, 48),
            nn.ReLU(True),
            nn.Linear(48, 55), 
            nn.Tanh())

    def forward(self, x):
#         x = self.conv1(x)
#         print('1.shape: ', x.shape)
#         x = self.pool(F.elu(self.bn2d1(x)))
#         print('2.shape: ', x.shape)
#         x = self.conv2(x)
#         x = self.pool(F.elu(self.bn2d2(x)))
#         x = self.conv3(x)
#         x = self.pool(F.elu(self.bn2d3(x)))
#         x = x.view(x.size(0), -1) # 9216
#         x = self.bn1d(self.fc1(x))
#         x = x.view(x.size(0), int(self.rep_dim / (4 * 4)), 4, 4)
#         x = F.elu(x)
#         x = self.deconv1(x)
#         x = F.interpolate(F.elu(self.bn2d4(x)), scale_factor=2)
#         x = self.deconv2(x)
#         x = F.interpolate(F.elu(self.bn2d5(x)), scale_factor=2)
#         x = self.deconv3(x)
#         x = F.interpolate(F.elu(self.bn2d6(x)), scale_factor=2)
#         x = self.deconv4(x)

        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [13]:
def load_dataset():
    """Loads the dataset.
    读取数据集， 要读取成"""
    
#     data  =  MSLSegLoader(data_path = '../input/msl-lzq', win_size = 100, step = 1, mode = 'train')
    data = CIFAR10_Dataset(root = '../input/msl-lzq')
    
    return data
# load_dataset



def build_network(net_name):
    """Builds the neural network."""

    net = CIFAR10_LeNet_ELU()

    return net


def build_autoencoder(net_name):
    """Builds the corresponding autoencoder network."""

    ae_net = CIFAR10_LeNet_ELU_Autoencoder()

    return ae_net



class Config(object):
    """Base class for experimental setting/configuration."""

    def __init__(self, settings):
        self.settings = settings

    def load_config(self, import_json):
        """Load settings dict from import_json (path/filename.json) JSON-file."""

        with open(import_json, 'r') as fp:
            settings = json.load(fp)

        for key, value in settings.items():
            self.settings[key] = value

    def save_config(self, export_json):
        """Save settings dict to export_json (path/filename.json) JSON-file."""

        with open(export_json, 'w') as fp:
            json.dump(self.settings, fp)


            
            

In [16]:
#  the following is the main function

#  记得： 记录一下每个参数的来源 ！！！！

import torch
import logging
import random
import numpy as np


def main(dataset_name, net_name, xp_path, data_path, load_config, load_model, objective, nu, device, seed, optimizer_name, lr, n_epochs, lr_milestone, batch_size, weight_decay, pretrain, ae_optimizer_name, ae_lr, ae_n_epochs, ae_lr_milestone, ae_batch_size, ae_weight_decay, n_jobs_dataloader, normal_class):
    
    """
    Deep SVDD, a fully deep method for anomaly detection.

    :arg DATASET_NAME: Name of the dataset to load.
    :arg NET_NAME: Name of the neural network to use.
    :arg XP_PATH: Export path for logging the experiment.
    :arg DATA_PATH: Root path of data.
    
    """

    # Get configuration
    cfg = Config(locals().copy())

    # Set up logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    log_file = xp_path + '/log.txt'
#     file_handler = logging.FileHandler(log_file)
#     file_handler.setLevel(logging.INFO)
#     file_handler.setFormatter(formatter)
#     logger.addHandler(file_handler)

#     # Print arguments
#     logger.info('Log file is %s.' % log_file)
#     logger.info('Data path is %s.' % data_path)
#     logger.info('Export path is %s.' % xp_path)

#     logger.info('Dataset: %s' % dataset_name)
#     logger.info('Normal class: %d' % normal_class)
#     logger.info('Network: %s' % net_name)

    # If specified, load experiment config from JSON-file
#     if load_config:
#         cfg.load_config(import_json=load_config)
#         logger.info('Loaded configuration from %s.' % load_config)

#     # Print configuration
#     logger.info('Deep SVDD objective: %s' % cfg.settings['objective'])
#     logger.info('Nu-paramerter: %.2f' % cfg.settings['nu'])

    # Set seed
    if cfg.settings['seed'] != -1:
        random.seed(cfg.settings['seed'])
        np.random.seed(cfg.settings['seed'])
        torch.manual_seed(cfg.settings['seed'])
        logger.info('Set seed to %d.' % cfg.settings['seed'])

    # Default device to 'cpu' if cuda is not available
    if not torch.cuda.is_available():
        device = 'cpu'
    logger.info('Computation device: %s' % device)
    logger.info('Number of dataloader workers: %d' % n_jobs_dataloader)

    # Load data
    dataset = load_dataset()
#     dataset = MSLSegLoader(data_path = '../input/msl-lzq', win_size = 100, step = 1, mode = 'train')
    # Initialize DeepSVDD model and set neural network \phi
    deep_SVDD = DeepSVDD(cfg.settings['objective'], cfg.settings['nu'])
    deep_SVDD.set_network(net_name)
    
    # If specified, load Deep SVDD model (radius R, center c, network weights, and possibly autoencoder weights)
    if load_model:
        deep_SVDD.load_model(model_path=load_model, load_ae=True)
        logger.info('Loading model from %s.' % load_model)

    logger.info('Pretraining: %s' % pretrain)
    if pretrain:
        # Log pretraining details
        logger.info('Pretraining optimizer: %s' % cfg.settings['ae_optimizer_name'])
        logger.info('Pretraining learning rate: %g' % cfg.settings['ae_lr'])
        logger.info('Pretraining epochs: %d' % cfg.settings['ae_n_epochs'])
        logger.info('Pretraining learning rate scheduler milestones: %s' % (cfg.settings['ae_lr_milestone'],))
        logger.info('Pretraining batch size: %d' % cfg.settings['ae_batch_size'])
        logger.info('Pretraining weight decay: %g' % cfg.settings['ae_weight_decay'])

        # Pretrain model on dataset (via autoencoder)
        deep_SVDD.pretrain(dataset,
                           optimizer_name=cfg.settings['ae_optimizer_name'],
                           lr=cfg.settings['ae_lr'],
                           n_epochs=cfg.settings['ae_n_epochs'],
                           lr_milestones=cfg.settings['ae_lr_milestone'],
                           batch_size=cfg.settings['ae_batch_size'],
                           weight_decay=cfg.settings['ae_weight_decay'],
                           device=device,
                           n_jobs_dataloader=n_jobs_dataloader)

    # Log training details
    logger.info('Training optimizer: %s' % cfg.settings['optimizer_name'])
    logger.info('Training learning rate: %g' % cfg.settings['lr'])
    logger.info('Training epochs: %d' % cfg.settings['n_epochs'])
    logger.info('Training learning rate scheduler milestones: %s' % (cfg.settings['lr_milestone'],))
    logger.info('Training batch size: %d' % cfg.settings['batch_size'])
    logger.info('Training weight decay: %g' % cfg.settings['weight_decay'])

    # Train model on dataset
    deep_SVDD.train(dataset,
                    optimizer_name=cfg.settings['optimizer_name'],
                    lr=cfg.settings['lr'],
                    n_epochs=cfg.settings['n_epochs'],
                    lr_milestones=cfg.settings['lr_milestone'],
                    batch_size=cfg.settings['batch_size'],
                    weight_decay=cfg.settings['weight_decay'],
                    device=device,
                    n_jobs_dataloader=n_jobs_dataloader)

    # Test model
    deep_SVDD.test(dataset, device=device, n_jobs_dataloader=n_jobs_dataloader)

    # Plot most anomalous and most normal (within-class) test samples
    '''这里把indices 给删了， 为不知道是干啥用的'''
#     indices, labels, scores = zip(*deep_SVDD.results['test_scores'])
    labels, scores = zip(*deep_SVDD.results['test_scores'])
#     indices, labels, scores = np.array(indices), np.array(labels), np.array(scores)
    scores = np.array(labels), np.array(scores)
#     idx_sorted = indices[labels == 0][np.argsort(scores[labels == 0])]  # sorted from lowest to highest anomaly score

#     if dataset_name in ('mnist', 'cifar10'):

#         if dataset_name == 'mnist':
#             X_normals = dataset.test_set.test_data[idx_sorted[:32], ...].unsqueeze(1)
#             X_outliers = dataset.test_set.test_data[idx_sorted[-32:], ...].unsqueeze(1)

#         if dataset_name == 'cifar10':
#             X_normals = torch.tensor(np.transpose(dataset.test_set.test_data[idx_sorted[:32], ...], (0, 3, 1, 2)))
#             X_outliers = torch.tensor(np.transpose(dataset.test_set.test_data[idx_sorted[-32:], ...], (0, 3, 1, 2)))

    # Save results, model, and configuration
    deep_SVDD.save_results(export_json=xp_path + '/results.json')
    deep_SVDD.save_model(export_model=xp_path + '/model.tar')
    cfg.save_config(export_json=xp_path + '/config.json')


if __name__ == '__main__':
    main(dataset_name = "cifar10", 
         net_name = 'Prof.Fan', 
         xp_path = '.', 
         data_path = '../input/msl-lzq/MSL_test.npy', 
         load_config = False, 
         load_model = False, 
         objective = 'soft-boundary', 
         nu = 0.1, 
         device = 'cuda', 
         seed = 1, 
         optimizer_name = 'adam', 
         lr = 0.001, 
         n_epochs = 5, 
         lr_milestone = (), 
         batch_size = 128, 
         weight_decay = 1e-6, 
         pretrain = True, 
         ae_optimizer_name = 'adam', 
         ae_lr = 0.001, 
         ae_n_epochs = 5, 
         ae_lr_milestone = (), 
         ae_batch_size = 128, 
         ae_weight_decay = 1e-6, 
         n_jobs_dataloader = 0, 
         normal_class = 0)

    
print('done')

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


lables:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
scores:  [1.65206592e+03 5.70078735e+02 6.24780273e+02 ... 4.35331494e-01
 1.69170020e+03 9.79591553e+02]
done


In [ ]:
import pandas as pd
import numpy as np
import json
import torch
def read_data(address = "../input/msl-lzq/MSL_test.npy"):
     return np.load(address)

file = read_data()

In [ ]:

import torch
from torch.autograd import Variable
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
print(file.shape)
int(np.prod(file.shape))
# file = torch.from_numpy(file)
file[0:10]

In [ ]:
from base.base_trainer import BaseTrainer
from base.base_dataset import BaseADDataset
from base.base_net import BaseNet
from torch.utils.data.dataloader import DataLoader
from sklearn.metrics import roc_auc_score

import logging
import time
import torch
import torch.optim as optim
import numpy as np

In [ ]:
from dsvdd import DeepSVDD


In [ ]:

def global_contrast_normalization(x, scale = 'l1'):
    assert scale in ('l1', 'l2')
    n_features = int(np.prod(x.shape)) # 所有维度特征数量和 n * w * h
    mean = torch.mean(x)
    x -= mean
    if scale == 'l1':
        x_scale = torch.mean(torch.abs(x)) 
    if scale == 'l2':
        x_scale = torch.sqrt(torch.sum( x**2 )) / n_features 
    x /= x_scale
    return x

data = global_contrast_normalization(file)

In [ ]:
data.shape

In [ ]:
def init_center_c(self, train_loader, net, eps = 0.1):
        n_samples = 0
        c = torch.zeros(net.rep_dim, device = self.device)
        net.eval()
        with torch.no_grad():
            for data in train_loader:
                inputs, _ = data
                inputs = inputs.to(self.device)
                outputs = net(inputs)
                n_samples += outputs.shape[0]
                c += torch.sum(outputs, dim = 0)
        c /= n_samples

        c[(abs(c) < eps) & (c < 0)] = -eps 
        c[(abs(c) > eps) & (c > 0)] = eps
        return c
